<a href="https://colab.research.google.com/github/Baek-Donghyeon/Bioinformatics-Algorithms/blob/main/Bioinformatics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bioinformatics

In [ ]:
import numpy as np
import random
import re
import collections
import itertools
from functools import reduce
import copy

from google.colab import drive
drive.mount('/content/drive')

gen_code = {'AAA':'K','AAC':'N','AAG':'K','AAU':'N','ACA':'T','ACC':'T','ACG':'T','ACU':'T',
            'AGA':'R','AGC':'S','AGG':'R','AGU':'S','AUA':'I','AUC':'I','AUG':'M','AUU':'I',
            'CAA':'Q','CAC':'H','CAG':'Q','CAU':'H','CCA':'P','CCC':'P','CCG':'P','CCU':'P',
            'CGA':'R','CGC':'R','CGG':'R','CGU':'R','CUA':'L','CUC':'L','CUG':'L','CUU':'L',
            'GAA':'E','GAC':'D','GAG':'E','GAU':'D','GCA':'A','GCC':'A','GCG':'A','GCU':'A',
            'GGA':'G','GGC':'G','GGG':'G','GGU':'G','GUA':'V','GUC':'V','GUG':'V','GUU':'V',
            'UAA':'*','UAC':'Y','UAG':'*','UAU':'Y','UCA':'S','UCC':'S','UCG':'S','UCU':'S',
            'UGA':'*','UGC':'C','UGG':'W','UGU':'C','UUA':'L','UUC':'F','UUG':'L','UUU':'F'}

int_mass = {'G':57,'A':71,'S':87,'P':97,'V':99,'T':101,'C':103,'I':113,'L':113,'N':114,
            'D':115,'K':128,'Q':128,'E':129,'M':131,'H':137,'F':147,'R':156,'Y':163,'W':186}

red_mass = {'G':57,'A':71,'S':87,'P':97,'V':99,'T':101,'C':103,'I':113,'N':114,
            'D':115,'K':128,'E':129,'M':131,'H':137,'F':147,'R':156,'Y':163,'W':186}

BLOSUM62 = {'A':{'A':4,'C':0,'D':-2,'E':-1,'F':-2,'G':0,'H':-2,'I':-1,'K':-1,'L':-1,'M':-1,'N':-2,'P':-1,'Q':-1,'R':-1,'S':1,'T':0,'V':0,'W':-3,'Y':-2,},
            'C':{'A':0,'C':9,'D':-3,'E':-4,'F':-2,'G':-3,'H':-3,'I':-1,'K':-3,'L':-1,'M':-1,'N':-3,'P':-3,'Q':-3,'R':-3,'S':-1,'T':-1,'V':-1,'W':-2,'Y':-2,},
            'D':{'A':-2,'C':-3,'D':6,'E':2,'F':-3,'G':-1,'H':-1,'I':-3,'K':-1,'L':-4,'M':-3,'N':1,'P':-1,'Q':0,'R':-2,'S':0,'T':-1,'V':-3,'W':-4,'Y':-3,},
            'E':{'A':-1,'C':-4,'D':2,'E':5,'F':-3,'G':-2,'H':0,'I':-3,'K':1,'L':-3,'M':-2,'N':0,'P':-1,'Q':2,'R':0,'S':0,'T':-1,'V':-2,'W':-3,'Y':-2,},
            'F':{'A':-2,'C':-2,'D':-3,'E':-3,'F':6,'G':-3,'H':-1,'I':0,'K':-3,'L':0,'M':0,'N':-3,'P':-4,'Q':-3,'R':-3,'S':-2,'T':-2,'V':-1,'W':1,'Y':3,},
            'G':{'A':0,'C':-3,'D':-1,'E':-2,'F':-3,'G':6,'H':-2,'I':-4,'K':-2,'L':-4,'M':-3,'N':0,'P':-2,'Q':-2,'R':-2,'S':0,'T':-2,'V':-3,'W':-2,'Y':-3,},
            'H':{'A':-2,'C':-3,'D':-1,'E':0,'F':-1,'G':-2,'H':8,'I':-3,'K':-1,'L':-3,'M':-2,'N':1,'P':-2,'Q':0,'R':0,'S':-1,'T':-2,'V':-3,'W':-2,'Y':2,},
            'I':{'A':-1,'C':-1,'D':-3,'E':-3,'F':0,'G':-4,'H':-3,'I':4,'K':-3,'L':2,'M':1,'N':-3,'P':-3,'Q':-3,'R':-3,'S':-2,'T':-1,'V':3,'W':-3,'Y':-1,},
            'K':{'A':-1,'C':-3,'D':-1,'E':1,'F':-3,'G':-2,'H':-1,'I':-3,'K':5,'L':-2,'M':-1,'N':0,'P':-1,'Q':1,'R':2,'S':0,'T':-1,'V':-2,'W':-3,'Y':-2,},
            'L':{'A':-1,'C':-1,'D':-4,'E':-3,'F':0,'G':-4,'H':-3,'I':2,'K':-2,'L':4,'M':2,'N':-3,'P':-3,'Q':-2,'R':-2,'S':-2,'T':-1,'V':1,'W':-2,'Y':-1,},
            'M':{'A':-1,'C':-1,'D':-3,'E':-2,'F':0,'G':-3,'H':-2,'I':1,'K':-1,'L':2,'M':5,'N':-2,'P':-2,'Q':0,'R':-1,'S':-1,'T':-1,'V':1,'W':-1,'Y':-1,},
            'N':{'A':-2,'C':-3,'D':1,'E':0,'F':-3,'G':0,'H':1,'I':-3,'K':0,'L':-3,'M':-2,'N':6,'P':-2,'Q':0,'R':0,'S':1,'T':0,'V':-3,'W':-4,'Y':-2,},
            'P':{'A':-1,'C':-3,'D':-1,'E':-1,'F':-4,'G':-2,'H':-2,'I':-3,'K':-1,'L':-3,'M':-2,'N':-2,'P':7,'Q':-1,'R':-2,'S':-1,'T':-1,'V':-2,'W':-4,'Y':-3,},
            'Q':{'A':-1,'C':-3,'D':0,'E':2,'F':-3,'G':-2,'H':0,'I':-3,'K':1,'L':-2,'M':0,'N':0,'P':-1,'Q':5,'R':1,'S':0,'T':-1,'V':-2,'W':-2,'Y':-1,},
            'R':{'A':-1,'C':-3,'D':-2,'E':0,'F':-3,'G':-2,'H':0,'I':-3,'K':2,'L':-2,'M':-1,'N':0,'P':-2,'Q':1,'R':5,'S':-1,'T':-1,'V':-3,'W':-3,'Y':-2,},
            'S':{'A':1,'C':-1,'D':0,'E':0,'F':-2,'G':0,'H':-1,'I':-2,'K':0,'L':-2,'M':-1,'N':1,'P':-1,'Q':0,'R':-1,'S':4,'T':1,'V':-2,'W':-3,'Y':-2,},
            'T':{'A':0,'C':-1,'D':-1,'E':-1,'F':-2,'G':-2,'H':-2,'I':-1,'K':-1,'L':-1,'M':-1,'N':0,'P':-1,'Q':-1,'R':-1,'S':1,'T':5,'V':0,'W':-2,'Y':-2,},
            'V':{'A':0,'C':-1,'D':-3,'E':-2,'F':-1,'G':-3,'H':-3,'I':3,'K':-2,'L':1,'M':1,'N':-3,'P':-2,'Q':-2,'R':-3,'S':-2,'T':0,'V':4,'W':-3,'Y':-1,},
            'W':{'A':-3,'C':-2,'D':-4,'E':-3,'F':1,'G':-2,'H':-2,'I':-3,'K':-3,'L':-2,'M':-1,'N':-4,'P':-4,'Q':-2,'R':-3,'S':-3,'T':-2,'V':-3,'W':11,'Y':2,},
            'Y':{'A':-2,'C':-2,'D':-3,'E':-2,'F':3,'G':-3,'H':2,'I':-1,'K':-2,'L':-1,'M':-1,'N':-2,'P':-3,'Q':-1,'R':-2,'S':-2,'T':-2,'V':-1,'W':2,'Y':7,}}

PAM250 = {'A':{'A':2,'C':-2,'D':0,'E':0,'F':-3,'G':1,'H':-1,'I':-1,'K':-1,'L':-2,'M':-1,'N':0,'P':1,'Q':0,'R':-2,'S':1,'T':1,'V':0,'W':-6,'Y':-3,},
          'C':{'A':-2,'C':12,'D':-5,'E':-5,'F':-4,'G':-3,'H':-3,'I':-2,'K':-5,'L':-6,'M':-5,'N':-4,'P':-3,'Q':-5,'R':-4,'S':0,'T':-2,'V':-2,'W':-8,'Y':0,},
          'D':{'A':0,'C':-5,'D':4,'E':3,'F':-6,'G':1,'H':1,'I':-2,'K':0,'L':-4,'M':-3,'N':2,'P':-1,'Q':2,'R':-1,'S':0,'T':0,'V':-2,'W':-7,'Y':-4,},
          'E':{'A':0,'C':-5,'D':3,'E':4,'F':-5,'G':0,'H':1,'I':-2,'K':0,'L':-3,'M':-2,'N':1,'P':-1,'Q':2,'R':-1,'S':0,'T':0,'V':-2,'W':-7,'Y':-4,},
          'F':{'A':-3,'C':-4,'D':-6,'E':-5,'F':9,'G':-5,'H':-2,'I':1,'K':-5,'L':2,'M':0,'N':-3,'P':-5,'Q':-5,'R':-4,'S':-3,'T':-3,'V':-1,'W':0,'Y':7,},
          'G':{'A':1,'C':-3,'D':1,'E':0,'F':-5,'G':5,'H':-2,'I':-3,'K':-2,'L':-4,'M':-3,'N':0,'P':0,'Q':-1,'R':-3,'S':1,'T':0,'V':-1,'W':-7,'Y':-5,},
          'H':{'A':-1,'C':-3,'D':1,'E':1,'F':-2,'G':-2,'H':6,'I':-2,'K':0,'L':-2,'M':-2,'N':2,'P':0,'Q':3,'R':2,'S':-1,'T':-1,'V':-2,'W':-3,'Y':0,},
          'I':{'A':-1,'C':-2,'D':-2,'E':-2,'F':1,'G':-3,'H':-2,'I':5,'K':-2,'L':2,'M':2,'N':-2,'P':-2,'Q':-2,'R':-2,'S':-1,'T':0,'V':4,'W':-5,'Y':-1,},
          'K':{'A':-1,'C':-5,'D':0,'E':0,'F':-5,'G':-2,'H':0,'I':-2,'K':5,'L':-3,'M':0,'N':1,'P':-1,'Q':1,'R':3,'S':0,'T':0,'V':-2,'W':-3,'Y':-4,},
          'L':{'A':-2,'C':-6,'D':-4,'E':-3,'F':2,'G':-4,'H':-2,'I':2,'K':-3,'L':6,'M':4,'N':-3,'P':-3,'Q':-2,'R':-3,'S':-3,'T':-2,'V':2,'W':-2,'Y':-1,},
          'M':{'A':-1,'C':-5,'D':-3,'E':-2,'F':0,'G':-3,'H':-2,'I':2,'K':0,'L':4,'M':6,'N':-2,'P':-2,'Q':-1,'R':0,'S':-2,'T':-1,'V':2,'W':-4,'Y':-2,},
          'N':{'A':0,'C':-4,'D':2,'E':1,'F':-3,'G':0,'H':2,'I':-2,'K':1,'L':-3,'M':-2,'N':2,'P':0,'Q':1,'R':0,'S':1,'T':0,'V':-2,'W':-4,'Y':-2,},
          'P':{'A':1,'C':-3,'D':-1,'E':-1,'F':-5,'G':0,'H':0,'I':-2,'K':-1,'L':-3,'M':-2,'N':0,'P':6,'Q':0,'R':0,'S':1,'T':0,'V':-1,'W':-6,'Y':-5,},
          'Q':{'A':0,'C':-5,'D':2,'E':2,'F':-5,'G':-1,'H':3,'I':-2,'K':1,'L':-2,'M':-1,'N':1,'P':0,'Q':4,'R':1,'S':-1,'T':-1,'V':-2,'W':-5,'Y':-4,},
          'R':{'A':-2,'C':-4,'D':-1,'E':-1,'F':-4,'G':-3,'H':2,'I':-2,'K':3,'L':-3,'M':0,'N':0,'P':0,'Q':1,'R':6,'S':0,'T':-1,'V':-2,'W':2,'Y':-4,},
          'S':{'A':1,'C':0,'D':0,'E':0,'F':-3,'G':1,'H':-1,'I':-1,'K':0,'L':-3,'M':-2,'N':1,'P':1,'Q':-1,'R':0,'S':2,'T':1,'V':-1,'W':-2,'Y':-3,},
          'T':{'A':1,'C':-2,'D':0,'E':0,'F':-3,'G':0,'H':-1,'I':0,'K':0,'L':-2,'M':-1,'N':0,'P':0,'Q':-1,'R':-1,'S':1,'T':3,'V':0,'W':-5,'Y':-3,},
          'V':{'A':0,'C':-2,'D':-2,'E':-2,'F':-1,'G':-1,'H':-2,'I':4,'K':-2,'L':2,'M':2,'N':-2,'P':-1,'Q':-2,'R':-2,'S':-1,'T':0,'V':4,'W':-6,'Y':-2,},
          'W':{'A':-6,'C':-8,'D':-7,'E':-7,'F':0,'G':-7,'H':-3,'I':-5,'K':-3,'L':-2,'M':-4,'N':-4,'P':-6,'Q':-5,'R':2,'S':-2,'T':-5,'V':-6,'W':17,'Y':0,},
          'Y':{'A':-3,'C':0,'D':-4,'E':-4,'F':7,'G':-5,'H':0,'I':-1,'K':-4,'L':-1,'M':-2,'N':-2,'P':-5,'Q':-4,'R':-4,'S':-3,'T':-3,'V':-2,'W':0,'Y':10,}}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Chapter 1

In [ ]:
def k_mer_of(text, k):

  for i in range(len(text) - k + 1):
    yield text[i:i+k]


def ham(pat, text):

  k = len(pat)
  dis = k

  for k_mer in k_mer_of(text, k):
    dis = min(dis, sum(pat[j] != k_mer[j] for j in range(k)))
  
  return dis


def pat_to_num(pat):

  sym_to_num = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

  if len(pat) == 0:
      return 0
  return 4*pat_to_num(pat[:-1]) + sym_to_num[pat[-1]]


def num_to_pat(num, k):

  num_to_sym = {0: 'A', 1: 'C', 2: 'G', 3: 'T'}

  try:
    if k == 1:
      return num_to_sym(num)
    return num_to_pat(num // 4, k-1) + num_to_sym[num % 4]
  except:
    raise ValueError()


def count_pat(text, pat):

  return sum(k_mer == pat for k_mer in k_mer_of(text, len(pat)))


def find_freq_k_mer(text, k, dis = 0, re = False):

  freq_dict = dict(freq_dict(text, k, dis, re))
  max = max(freq_dict, key = lambda x : freq_dict[x])

  return list(filter(lambda x : x[1] == max, freq_dict.items())) 


def freq_dict(text, k, dis = 0, re = False):
  
  if re:
    return freq_dict(re_complement(text), k, dis) + freq_dict(text, k, dis)
  else:
    return collections.Counter([neigh for k_mer in k_mer_of(text, k) for neigh in neigh(k_mer, dis)])


def re_complement(text):

  compliment_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
  return ''.join(reversed([compliment_dict[base] for base in text]))


def find_pat_occur(text, pat, dis = 0):

  # ham(text, pat) ≤ d
  k = len(pat)
  return [i for i in range(len(text) - k + 1) if ham(text[i:i+k], pat) <= dis]


def find_clump(text, k, L, t):

  '''
  k   length of the k_mer
  L   length of an interval
  t   min ocurrences of a pat in the interval
  '''

  pos_dict = collections.defaultdict(list)
  freq_dict = freq_dict(text, k)
  for pat, freq in freq_dict.items():
    if freq >= t:
      pos_dict[pat] = find_occurrences_of_pat(text, pat)
  for pat, pos in pos_dict.items():
    for i in range(len(pos) - t + 1):
      if pos[i+t-1] - pos[i] <= L - k + 1:
        yield pat


def find_min_skew_pos(text):

  # skew : abs(#'G'-#'C')
  skew = [0]
  for base in text:
    if base == 'C': skew.append(skew[-1]-1)
    elif base == 'G': skew.append(skew[-1]+1)
    else: skew.append(skew[-1])
  min = min(skew)
  return list(filter(lambda x : x == min, skew)) # return minimum skew


def neigh(pat, dis):

  length = len(pat)
  mis_base = itertools.product("ACTG", repeat = dis) # all possible 'dis' length letter strings
  for poss in itertools.combinations(iter(range(length)), dis):
    for base in mis_base:
      neigh = copy.deepcopy(pat)
      for i in dis:
        neigh[pos[i]] = base[i]
        yield neigh

Chapter 2

In [ ]:
def enum_motif(strings, k, dis):

  # find all (k, dis)-motifs in a collect of strings.
  neigh_list = []
  for text in strings:
    neigh_list.append(set(neigh(text, k, dis)))
  motifs = neigh_list[0]
  for neigh in neigh_list:
    motifs = motifs & neigh
  return list(motifs)


def find_median_string(strings, k, dis = 0):

  # a median string for Dna minimizes d(pat, Dna) over all k-mers pat.
  motifs = enum_motif(strings, k, dis)
  if not motifs:
    find_median_string(strings, k, dis + 1)
  else:
    motifs_dict = collections.defaultdict(int)
    for pat in motifs:
      motifs_dict[pat] = sum(ham(pat, text) for text in strings)
    return min(motifs_dict, key = lambda x : motifs_dict[x]) # return key that minimizes the value


def find_prob_k_mer(text, k, profile):

  # A k-mer that was most likely to have been generated by Profile among all k-mers in Text.
  best_score = 0
  for k_mer in k_mer_of(text, k):
    s = dna_score(profile, [k_mer])
    if best_score < s:
      best_score = s
      prob_k_mer = k_mer
  return prob_k_mer


def dna_score(profile, motif):

  str_len = len(motif[0])
  return sum(reduce(lambda x, y: x*y, [profile[pat_to_num(k_mer[i])][i]
              for i in range(str_len)]) for k_mer in motif)


def form_profile(motif):

  k = len(motif[0])
  profile = np.full((4,k), 1) # pseudocount
  for i in range(k):
    for k_mer in motif:
      profile[pat_to_num(k_mer[i])][i] += 1
  return profile/(len(motif)+4)


def greedy_motif_search(strings, k):

  best_motif = [text[:k] for text in strings]
  for k_mer in k_mer_of(strings[0], k):
    motif = [k_mer]
    for j in range(1,len(strings)):
      profile = form_profile(motif)
      motif.append(find_prob_k_mer(strings[j], k, profile))
    if dna_score(form_profile(best_motif), best_motif) < dna_score(form_profile(motif), motif):
      best_motif = motif
  return best_motif


def gibbs_sampler(strings, k, n):
  str_len = len(strings[0])
  t = len(strings)
  best_motif_score = 0

  for time in range(n):

    # generate random motif
    motif = []
    for i in range(t):
      motif.append(random.choice([k_mer for k_mer in k_mer_of(strings[i], k)]))

    # update the motif the profile matches best
    last = None
    while motif != last:
      last = motif
      for i in range(t):
        motif[i] = find_prob_k_mer(strings[i], k, form_profile([motif[j] for j in range(t) if j != i]))
    
    motif_score = dna_score(form_profile(motif), motif)

    if best_motif_score < motif_score:
      best_motif_score = motif_score
      best_motif = motif
      print(best_motif_score)

  return best_motif


def rand_motif_search(strings, k):

  str_len = len(strings[0])
  t = len(strings)
  best_motif_score = 0 # score
  
  for time in xrange(800):

    # generate random motif
    motif = []
    for i in range(t):
      motif.append(random.choice([k_mer for k_mer in k_mer_of(strings[i], k)]))
    better_motif = motif

    # converge to a local optimum
    while True:
      profile = form_profile(better_motif)
      motif = []
      for j in range(t):
        motif.append(find_prob_k_mer(strings[j], k, profile))
        motif_score = dna_score(profile, better_motif)
      if motif_score < dna_score(form_profile(motif), motif):
        better_motif = motif
      else:
        break

    if best_motif_score < motif_score:
      best_motif_score = motif_score
      best_motif = better_motif

  return best_motif

Chapter 3

In [ ]:
def find_string(strings, d = None):

  first = strings[0]

  if d != None: # paired
    half = len(first)//2
    return find_string([front[:half] for front in strings]) + find_string([rear[half:] for rear in strings[-half-d:]])

  return first[:-1] + ''.join([text[-1] for text in strings]) 


def de_brujin_graph(strings):

  graph = collections.defaultdict(list)

  if len(strings[0]) == 2: # paired
    for f,r in strings: # ex) GAGA|TTGA, f = GAGA, r = TTGA
      graph[f[:-1]+r[:-1]].append(f[1:]+r[1:])         
  else: # unpaired
    for k_mer in strings:
      graph[k_mer[:-1]].append(k_mer[1:])
  return graph


def eulerian(graph):

  # determine whether it is an eulerian path or cycle
  graph = copy.deepcopy(graph)
  in_degree = collections.Counter(list(itertools.chain(*graph.values())))
  out_degree = collections.Counter([k for k,v in graph.items() for __ in range(len(v))])
  odd_degree = (out_degree - in_degree).keys()
  if odd_degree:
    stack = list(odd_degree) # start from odd_degree / eulerian path
  else:
    stack = [list(graph.keys())[0]] # start from arbitrary node / eulerian cycle

  # DFS
  visit = []
  while stack:
    try:
      stack.append(graph[stack[-1]].pop())
    except: # backtrack
      visit.append(stack.pop())
  visit.reverse()
  return visit


def reconstruct_string(strings, d = None):

  # d = # of bases b/w front and rear ends
  return find_string(eulerian(de_brujin_graph(strings)),d)


def find_k_universal_circular_string(k):

  # [:-k+1] : for circular_string
  return reconstruct_string(list(map(''.join, product('01', repeat=k))))[:-k+1]


def generate_contig(strings):

  graph = de_brujin_graph(strings)

  # branch : in != 1 & out != 1
  in_degree = collections.Counter(list(itertools.chain(*graph.values())))
  out_degree = collections.Counter([k for k,v in graph.items() for __ in range(len(v))])
  branch = set(node for node in in_degree.keys() if in_degree[node] != 1)
  branch = branch & set(node for values in graph.values() for node in values if out_degree[node] != 1)
  
  # append node until branch
  collect = []
  for node in branch:
    while graph[node]:
      contig = [node, graph[node].pop()]
      while contig[-1] not in branch:
        contig.append(graph[contig[-1]].pop())
      collect.append(find_string(contig))
  return collect

Chapter 4

In [ ]:
def find_encoding_string(text, pep):

  def dna_to_rna(text):
    return re.sub('T','U',text)
  
  def translate(text):
    aa = []
    for i in range(0,len(text),3):
      codon = text[i:i+3]
      if codon in ['UAA','UAG','UGA']:
        break
      aa.append(gen_code[codon])
    return ''.join(aa)

  pep_len = len(pep)
  answer = []
  for i in range(len(text)-3*pep_len+1):
    string = text[i:i+3*pep_len]
    if translate(dna_to_rna(string)) == pep or translate(dna_to_rna(re_complement(string))) == pep:
      answer.append(string)
  return answer


def spectrum(pep, cyclo = False):

  pep_len = len(pep)
  spec = [0]

  if cyclo:
    spec.append(sum(pep))
    circle = pep+pep
    for i in range(pep_len):
      for j in range(1, pep_len):
        spec.append(sum(circle[i:i+j]))
    return sorted(spec)
  else: # linear
    for i in range(pep_len):
      for j in range(i, pep_len):
        spec.append(sum(pep[i:j+1]))
    return sorted(spec)


def mass(pep):
  return sum([int_mass[aa] for aa in pep])


def num_of_pep(m):
  d = collections.defaultdict(int)
  aa_mass = red_mass.values()
  d[0] = 1 # single amino acid
  for i in range(1,m+1):
    d[i] = sum([d[i-j] for j in aa_mass if i-j >= 0])
  return d[m]


def pep_score(pep, spec, cyclo = False):
  score = 0
  s = spectrum(pep, cyclo)
  # the scoring function takes into account the multiplicity
  for frag in spec:
    if frag in s:
      s.remove(frag)
      score += 1
  return score


def leaderboard_cyclo_seq(spec, n, m):

  def trim(leader_board, spec, n):
    # return the top m hiehgest-scoring peptides with ties
    l = []
    for pep in leader_board:
      l.append([pep_score(pep, spec),pep])

  def convolution(spec, n):

    convol = collections.defaultdict(int)
    answer = []

    # add differences of masses to convol
    for i in spec:
      for j in spec:
        if i-j > 0:
          convol[i-j] += 1
    
    l = sorted(convol.items(), key = lambda x : x[1], reverse=True)
    for i in range(n+1, len(l)): # top m elements
      if l[n][1] > l[i][1]:
        return [[k] for k,v in l[:i-1] if k in red_mass.values()]
        break

    return [[k] for k,v in l if k in red_mass.values()]

      l = sorted(l, key = lambda x : x[0], reverse=True)
      for i in range(n+1, len(l)): # top m elements
        if l[n][0] > l[i][0]:
          leader_board = [v for k,v in l[:i-1]]
          break

      return leader_board
    
  candidate = convolution(spec, n)
  leader_board = candidate
  leader_pep = []
  total_mass = max(spec)

  while True:
    l_b = copy.deepcopy(leader_board)
    for pep in l_b:
      sub_mass = sum(pep)
      if sub_mass == total_mass:
        if pep_score(pep, spec, cyclo=True) > pep_score(leader_pep, spec, cyclo=True):
          leader_pep = pep
      elif sub_mass > total_mass:
        leader_board.remove(pep)

    # break if leader_board is empty
    if not leader_board:
      break
    
    leader_board = trim(leader_board, spec, m)
    # add an a.a. to pep
    leader_board = [pep+aa for pep in leader_board for aa in candidate]

  return leader_pep


def turn_pike(L, A=[0]):

  L = copy.deepcopy(L)
  
  # remove pairwise differencese b/w points in A[:-1] and A[-1]
  for a in A[:-1]:
    L.remove(a - A[-1])
    L.remove(A[-1] - a)
  L.remove(0)

  if not L:
    return sorted(A)
  else:
    try:
      return turn_pike(L, A+[max(L)])
    except:
      return turn_pike(L, A+[max(A)-max(L)])

Chapter 5

In [ ]:
def find_min_num_coin(money, coin):

  min_num_coin = [0]*(money+1)
  for m in range(1,money+1):
    min_num_coin[m] = float('inf')
    for c in coin:
      if m >= c:
        min_num_coin[m] = min(min_num_coin[m-c] + 1, min_num_coin[m])
  return min_num_coin[-1]


def find_longest_path(down,right):
  n = len(right)
  m = len(down[0])
  s = [[0]*m for i in range(n)]
  for i in range(1,n):
    s[i][0] = s[i-1][0]+down[i-1][0]
  for j in range(1,m):
    s[0][j] = s[0][j-1]+right[0][j-1]
  for i in range(1,n):
    for j in range(1,m):
      s[i][j] = max(s[i-1][j]+down[i-1][j],s[i][j-1]+right[i][j-1])
  return s[-1][-1]


def longest_com_seq(v, w):
  
  n, m = len(v), len(w)
  s = [[0]*(m+1) for i in range(n+1)]
  track =[[0]*(m+1) for i in range(n+1)]

  for i in range(1,n+1):
    for j in range(1,m+1):
      s[i][j] = max(s[i-1][j], s[i][j-1], s[i-1][j-1] + (v[i-1] == w[j-1]))
      if s[i][j] == s[i-1][j]: track[i][j] = 'd' # delition (down)
      elif s[i][j] == s[i][j-1]: track[i][j] = 'i' # insertion (right)
      else: track[i][j] = 'm' # match

  def backtrack(track, v, i, j):
    if i == 0 or j == 0: return ''
    if track[i][j] == 'd': return backtrack(track, v, i-1, j)
    if track[i][j] == 'i': return backtrack(track, v, i, j-1)
    return backtrack(track, v, i-1, j-1) + v[i-1]

  return backtrack(track, v, n, m)


def longest_path(source, sink, DAG): # directed acyclic graph

  n = len(DAG)
  s = [0]*n
  track = [None]*n

  # sort out valid point that is accesible from the source
  valid = [source]
  for node in valid:
    valid.extend([i for i in range(node, n) if DAG[node][i] != None and i not in valid])
  valid.sort()

  for i in valid:
    for j in valid:
      if DAG[j][i] != None and s[i] < DAG[j][i] + s[j]:
        s[i] = DAG[j][i] + s[j]
        track[i] = j
  
  # backtrack
  path = [sink]
  while path[-1] != source:
    path.append(track[path[-1]])
  path.reverse()

  return path, s[sink]

def global_align(v,w,penalty):
  
  n, m = len(v), len(w)
  s = [[0]*(m+1) for i in range(n+1)]
  track =[[0]*(m+1) for i in range(n+1)]

  # initialize s matrix
  s[0] = [-penalty*i for i in range(m+1)]
  for i in range(n+1):
    s[i][0] = -penalty*i

  for i in range(1,n+1):
    for j in range(1,m+1):
      s[i][j] = max(s[i-1][j]-penalty, s[i][j-1]-penalty, s[i-1][j-1]+BLOSUM62[v[i-1]][w[j-1]])
      if s[i][j] == s[i-1][j]-penalty: track[i][j] = 'd' # delition (down)
      elif s[i][j] == s[i][j-1]-penalty: track[i][j] = 'i' # insertion (right)
      else: track[i][j] = 'm' # match

  def backtrack(track, i, j):
    if i == 0: return 'i'*j
    if j == 0: return 'd'*i
    if track[i][j] == 'd': return backtrack(track, i-1, j)+'d'
    if track[i][j] == 'i': return backtrack(track, i, j-1)+'i'
    return backtrack(track, i-1, j-1)+'m' # track[i][j] == 'm'

  # backtrack
  v_aligned,w_aligned = [],[]
  v,w = iter(v), iter(w)
  for c in backtrack(track, n, m):
    if c == 'd':
      v_aligned.append(next(v))
      w_aligned.append('-')
    elif c == 'i':
      v_aligned.append('-')
      w_aligned.append(next(w))
    else: # c == 'm'
      v_aligned.append(next(v))
      w_aligned.append(next(w))

  return  s[-1][-1], ''.join(v_aligned), ''.join(w_aligned)


def local_align(v,w,penalty,flag=None):
  
  n, m = len(v), len(w)
  s = [[0]*(m+1) for i in range(n+1)]
  track =[[0]*(m+1) for i in range(n+1)]

  for i in range(1,n+1):
    for j in range(1,m+1):
      s[i][j] = max(0,s[i-1][j]-penalty, s[i][j-1]-penalty, s[i-1][j-1]+PAM250[v[i-1]][w[j-1]])
      if s[i][j] == 0: track[i][j] = 0
      elif s[i][j] == s[i-1][j]-penalty: track[i][j] = 'd' # delition (down)
      elif s[i][j] == s[i][j-1]-penalty: track[i][j] = 'i' # insertion (right)
      else: track[i][j] = 'm' # match
  
  # find indices where s[i][j] is maximum (end-point of local alignment)
  local_max = 0
  for i in range(n+1):
    for j in range(m+1):
      if local_max < s[i][j]:
        local_max = s[i][j]
        end_i, end_j = i, j

  def backtrack(track, trail, i, j):
    if track[i][j] == 0: return i,j,trail # return the starting point
    if track[i][j] == 'd':
      trail += 'd'
      return backtrack(track, trail, i-1, j)
    if track[i][j] == 'i':
      trail += 'i'
      return backtrack(track, trail, i, j-1)
    trail += 'm'
    return backtrack(track, trail, i-1, j-1) # track[i][j] == 'm'

  # backtrack
  start_i, start_j, trail = backtrack(track, '', end_i, end_j)
  trail = trail[::-1]
  v_aligned,w_aligned = [],[]
  v,w = iter(v[start_i:end_i]), iter(w[start_j:end_j])
  for c in trail:
    if c == 'd':
      v_aligned.append(next(v))
      w_aligned.append('-')
    elif c == 'i':
      v_aligned.append('-')
      w_aligned.append(next(w))
    else: # c == 'm'
      v_aligned.append(next(v))
      w_aligned.append(next(w))

  return local_max, ''.join(v_aligned), ''.join(w_aligned)

def edit_dis(v,w):
  
  n, m = len(v), len(w)
  s = [[0]*(m+1) for i in range(n+1)]
  track =[[0]*(m+1) for i in range(n+1)]

  # initialize s matrix
  s[0] = [-i for i in range(m+1)]
  for i in range(n+1):
    s[i][0] = -i

  for i in range(1,n+1):
    for j in range(1,m+1):
      s[i][j] = max(s[i-1][j]-1, s[i][j-1]-1, s[i-1][j-1]+(v[i-1] == w[j-1])-1)

  return -s[-1][-1]


def fitting(v,w):

  n, m = len(v), len(w)
  s = [[0]*(m+1) for i in range(n+1)]
  track =[[0]*(m+1) for i in range(n+1)]

  # initialize s matrix
  s[0] = [-i for i in range(m+1)]

  for i in range(1,n+1):
    for j in range(1,m+1):
      s[i][j] = max(s[i-1][j]-1, s[i][j-1]-1, s[i-1][j-1]+2*(v[i-1]==w[j-1])-1)
      if s[i][j] == s[i-1][j]-1: track[i][j] = 'd' # delition (down)
      elif s[i][j] == s[i][j-1]-1: track[i][j] = 'i' # insertion (right)
      else: track[i][j] = 'm' # match

  # find indices where s[i][j] is maximum (end-point of local alignment)
  local_max = 0
  for i in range(n+1):
    if local_max < s[i][m]:
      local_max = s[i][m]
      end_i = i

  def backtrack(track, trail, i, j):
    if j == 0: return i,trail # return the starting point
    if i == 0:
      trail += 'i'*j
      return i, trail
    if track[i][j] == 'd':
      trail += 'd'
      return backtrack(track, trail, i-1, j)
    if track[i][j] == 'i':
      trail += 'i'
      return backtrack(track, trail, i, j-1)
    trail += 'm'
    return backtrack(track, trail, i-1, j-1) # track[i][j] == 'm'

  # backtrack
  start_i, trail = backtrack(track, '', end_i, m)
  trail = trail[::-1]
  v_aligned,w_aligned = [],[]
  v,w = iter(v[start_i:end_i]), iter(w)
  for c in trail:
    if c == 'd':
      v_aligned.append(next(v))
      w_aligned.append('-')
    elif c == 'i':
      v_aligned.append('-')
      w_aligned.append(next(w))
    else: # c == 'm'
      v_aligned.append(next(v))
      w_aligned.append(next(w))

  return local_max, ''.join(v_aligned), ''.join(w_aligned)

def overlap(v,w):

  n, m = len(v), len(w)
  s = [[0]*(m+1) for i in range(n+1)]
  track =[[0]*(m+1) for i in range(n+1)]

  # initialize s matrix
  s[0] = [-i for i in range(m+1)]

  for i in range(1,n+1):
    for j in range(1,m+1):
      s[i][j] = max(s[i-1][j]-2, s[i][j-1]-2, s[i-1][j-1]+3*(v[i-1]==w[j-1])-2)
      if s[i][j] == s[i-1][j]-2: track[i][j] = 'd' # delition (down)
      elif s[i][j] == s[i][j-1]-2: track[i][j] = 'i' # insertion (right)
      else: track[i][j] = 'm' # match

  # find indices where s[i][j] is maximum (end-point of local alignment)
  local_max = 0
  for j in range(m+1):
    if local_max < s[n][j]:
      local_max = s[n][j]
      end_j = j

  def backtrack(track, trail, i, j):
    if j == 0: return i,trail # return the starting point
    if i == 0:
      trail += 'i'*j
      return i, trail
    if track[i][j] == 'd':
      trail += 'd'
      return backtrack(track, trail, i-1, j)
    if track[i][j] == 'i':
      trail += 'i'
      return backtrack(track, trail, i, j-1)
    trail += 'm'
    return backtrack(track, trail, i-1, j-1) # track[i][j] == 'm'

  # backtrack
  start_i, trail = backtrack(track, '', n, end_j)
  trail = trail[::-1]
  v_aligned,w_aligned = [],[]
  v,w = iter(v[start_i:]), iter(w[:end_j])
  for c in trail:
    if c == 'd':
      v_aligned.append(next(v))
      w_aligned.append('-')
    elif c == 'i':
      v_aligned.append('-')
      w_aligned.append(next(w))
    else: # c == 'm'
      v_aligned.append(next(v))
      w_aligned.append(next(w))

  return local_max, ''.join(v_aligned), ''.join(w_aligned)

def affine_gap(v,w,gap=(11,1)):
  
  open, ext = gap # opening penalty, extension penalty
  n, m = len(v), len(w)
  # [0]:lower/delete, [1]:middle/match, [2]:upper/insert
  s = [[[0]*(m+1) for i in range(n+1)] for j in range(3)]
  track = [[[0]*(m+1) for i in range(n+1)] for j in range(3)]

  # initialize s matrix
  for i in range(3):
    s[i][0] = [0]+[-open-i*ext for j in range(m)]
    for k in range(n):
      s[i][k+1][0] = -open-k*ext

  for i in range(1,n+1):
    for j in range(1,m+1):
      s[0][i][j] = max(s[0][i-1][j]-ext,s[1][i-1][j]-open)
      s[2][i][j] = max(s[2][i][j-1]-ext,s[1][i][j-1]-open)
      s[1][i][j] = max(s[0][i][j], s[1][i-1][j-1]+BLOSUM62[v[i-1]][w[j-1]], s[2][i][j])
      track[0][i][j] = 'l' if s[0][i][j] == s[0][i-1][j]-ext else 'm'
      track[2][i][j] = 'u' if s[2][i][j] == s[2][i][j-1]-ext else 'm'
      if s[1][i][j] == s[0][i][j]: track[1][i][j] = 'l'
      elif s[1][i][j] == s[2][i][j]: track[1][i][j] = 'u'
      else: track[1][i][j] = 'm'

  def backtrack(track, i, j, level=1): # level 0:l, 1:m, 2:u
    if i == 0: return 'i'*j
    if j == 0: return 'd'*i
    if level == 0:
      return backtrack(track, i-1, j, 0 if track[level][i][j] == 'l' else 1) + 'd'
    elif level == 1:
      if track[level][i][j] == 'l': return backtrack(track, i, j, 0)
      elif track[level][i][j] == 'u': return backtrack(track, i, j, 2)
      else: return backtrack(track, i-1, j-1, 1)+'m' # track[i][j] == 'm'
    else: # level == 2
      return backtrack(track, i, j-1, 2 if track[level][i][j] == 'u' else 1) + 'i'

  # backtrack
  v_aligned,w_aligned = [],[]
  v,w = iter(v), iter(w)
  for c in backtrack(track, n, m):
    if c == 'd':
      v_aligned.append(next(v))
      w_aligned.append('-')
    elif c == 'i':
      v_aligned.append('-')
      w_aligned.append(next(w))
    else: # c == 'm'
      v_aligned.append(next(v))
      w_aligned.append(next(w))

  return  s[1][-1][-1], ''.join(v_aligned), ''.join(w_aligned)

def find_mid_edge(v,w,left,right,penalty):

  n, m = len(v), len(w)
  mid = m//2
  print(n,m,len(left),len(right))
  # from_source
  source = [[i,0] for i in left]
  source[0][1] = source[0][0] - penalty
  for i in range(1,n+1):
    source[i][1] = max(source[i-1][1]-penalty, source[i][0]-penalty, source[i-1][0]+BLOSUM62[v[i-1]][w[0]])
  
  for j in range(1,mid+1):
    for i in range(n+1):
      source[i][0] = source[i][1]
    source[0][1] = source[0][0] - penalty
    for i in range(1,n+1):
      source[i][1] = max(source[i-1][1]-penalty, source[i][0]-penalty, source[i-1][0]+BLOSUM62[v[i-1]][w[j]])

  # to_sink

  re_v, re_w = v[::-1], w[::-1]
  sink = [[i,0] for i in right]
  sink[0][1] = sink[0][0] - penalty
  for i in range(1,n+1):
    sink[i][1] = max(sink[i-1][1]-penalty, sink[i][0]-penalty, sink[i-1][0]+BLOSUM62[re_v[i-1]][re_w[0]])
  
  for j in range(1,m-mid):
    for i in range(n+1):
      sink[i][0] = sink[i][1]
    sink[0][1] = sink[0][0] - penalty
    for i in range(1,n+1):
      sink[i][1] = max(sink[i-1][1]-penalty, sink[i][0]-penalty, sink[i-1][0]+BLOSUM62[re_v[i-1]][re_w[j]])

  # find the middle node
  length = float('-inf')
  for i in range(n+1):
    if length < source[i][0] + sink[n-i][1]:
      length = source[i][0] + sink[n-i][1]
      start_y = i
  start = (start_y, mid)

  # find the middle edge
  length = float('-inf')
  for i in range(n+1):
    if length < source[i][1] + sink[n-i][0]:
      length = source[i][1] + sink[n-i][0]
      end_y = i
  end = (end_y, mid+1)

  return start, end, [i[1] for i in sink], [i[1] for i in source]


def multi_global_align(u, v, w):
  
  n, m, l = len(u), len(v), len(w)
  s = np.zeros((n+1,m+1,l+1))
  track = np.empty((n+1,m+1,l+1),dtype='str')

  for i in range(1,n+1):
    for j in range(1,m+1):
      for k in range(1,l+1):
        s[i][j][k] = max(s[i-1][j-1][k], s[i][j-1][k-1], s[i-1][j][k-1], s[i-1][j][k], s[i][j-1][k], s[i][j][k-1], s[i-1][j-1][k-1] + (u[i-1]==v[j-1] and v[j-1]==w[k-1]))
        if s[i][j][k] == s[i-1][j-1][k]: track[i][j][k] = 'K'
        elif s[i][j][k] == s[i][j-1][k-1]: track[i][j][k] = 'I'
        elif s[i][j][k] == s[i-1][j][k-1]: track[i][j][k] = 'J'
        elif s[i][j][k] == s[i-1][j][k]: track[i][j][k] = 'i'
        elif s[i][j][k] == s[i][j-1][k]: track[i][j][k] = 'j'
        elif s[i][j][k] == s[i][j][k-1]: track[i][j][k] = 'k'
        else: track[i][j][k] = 'm' # match

  def backtrack(track, i, j, k):
    if track[i][j][k] == 'm': return backtrack(track, i-1, j-1, k-1)+'m'
    elif track[i][j][k] == 'I': return backtrack(track, i, j-1, k-1)+'I'
    elif track[i][j][k] == 'J': return backtrack(track, i-1, j, k-1)+'J'
    elif track[i][j][k] == 'K': return backtrack(track, i-1, j-1, k)+'K'
    elif track[i][j][k] == 'i': return backtrack(track, i-1, j, k)+'i'
    elif track[i][j][k] == 'j': return backtrack(track, i, j-1, k)+'j'
    elif track[i][j][k] == 'k': return backtrack(track, i, j, k-1)+'k'
    else: return 'i'*i+'j'*j+'k'*k

  # backtrack
  u_aligned, v_aligned, w_aligned = [],[],[]
  u,v,w = iter(u), iter(v), iter(w)
  for c in backtrack(track, n, m, l):
    if c == 'I':
      u_aligned.append('-')
      v_aligned.append(next(v))
      w_aligned.append(next(w))
    elif c == 'J':
      u_aligned.append(next(u))
      v_aligned.append('-')
      w_aligned.append(next(w))
    elif c == 'K':
      u_aligned.append(next(u))
      v_aligned.append(next(v))
      w_aligned.append('-')
    elif c == 'i':
      u_aligned.append(next(u))
      v_aligned.append('-')
      w_aligned.append('-')
    elif c == 'j':
      u_aligned.append('-')
      v_aligned.append(next(v))
      w_aligned.append('-')
    elif c == 'k':
      u_aligned.append('-')
      v_aligned.append('-')
      w_aligned.append(next(w))      
    else: # c == 'm'
      u_aligned.append(next(u))
      v_aligned.append(next(v))
      w_aligned.append(next(w))

  return  s[-1][-1][-1], ''.join(u_aligned), ''.join(v_aligned), ''.join(w_aligned)

def topo_order(adj):
  Q = collections.deque()
  L = []
  while True:
    for key in adj.keys():
      if key not in list(itertools.chain(*adj.values())) and key not in Q:
        Q.append(key)
    node = Q.popleft()
    L.append(node)
    del adj[node]
    if not adj:
      break
  return L

Chapter 6

In [ ]:
def greedy_sorting(permutation):
  sequence = []
  for i in range(len(permutation)):
    if permutation[i] == i+1:
      continue
    else:
      for j in range(i,len(permutation)):
        if abs(permutation[j]) == i+1:
          permutation[i:j+1] = [-x for x in reversed(permutation[i:j+1])]
          sequence.append(copy.deepcopy(permutation))
    if permutation[i] == -i-1:
      permutation[i] *= -1
      sequence.append(copy.deepcopy(permutation))
  return sequence

def break_point(permutation):
  count = 0
  permutation = [0] + permutation + [len(permutation)+1]
  for i in range(len(permutation)-1):
    if permutation[i]+1 != permutation[i+1]:
      count += 1
  return count


def chrom_to_cycle(chromosome):
  node = []
  for n in chromosome:
    if n > 0:
      node.extend([2*n-1, 2*n])
    else:
      node.extend([-2*n, -2*n-1])
  return node


def cycle_to_chrom(node):
  chromosome = []
  for i in range(len(node)//2):
    if node[2*i] < node[2*i+1]:
      chromosome.append((node[2*i]+1)//2)
    else:
      chromosome.append(-node[2*i]//2)
  return chromosome

def color_edge(genome):
  edge = []
  for chromosome in genome:
    node = chrom_to_cycle(chromosome)
    for i in range(len(chromosome)):
      edge.append((node[2*i-1],node[2*i]))
  return edge


def graph_to_genome(graph):

  temp = []
  nodes = []
  for node in graph:
    temp.append(node)
    if node[0] > node[1]:
      nodes.append(temp)
      temp = []
  
  genome = []
  for node in nodes:
    cycle = []
    for i, j in node:
      cycle.extend([i, j])
    genome.append(cycle_to_chrom(cycle[-1:]+cycle[:-1]))

  return genome


Chapter 7

In [ ]:
def leave_distance(n, adj):

  dis = np.zeros((n, n))

  # finding path: root -> goal
  for root in range(n):
    for goal in range(root, n):
      # bfs
      Q = collections.deque([root])
      parant = [-1] * len(adj[0])
      while Q:
        v = Q.popleft()
        if v == goal:
          # add up all the lengths along the path
          length = 0
          child = goal
          while child != root:
            length += adj[parant[child], child]
            child = parant[child]
          dis[root, goal] = length
          dis[goal, root] = length
          break
        else:
          for w in np.nonzero(adj[v])[0]:
            if parant[w] == -1:
              Q.append(w)
              parant[w] = v
  return dis.astype(int)


def limb(dis, j):

  n = len(dis[0])
  best_l = float('inf')
  for i in range(n):
    for k in range(n):
      l = (dis[j][k] + dis[j][i] - dis[i][k])/2
      if 0 < l and l < best_l:
        best_l = l
  return int(best_l)


def nearest(T, l, i, k):

  '''
  find the nearest two nodes on path i->k
  T : Tree
  l : distance from i to new node
  i, k : leaves
  '''

  # bfs, k->i
  Q = collections.deque([k])
  parant = [-1] * len(T[0])
  while Q:
    v = Q.popleft()
    if v == i:
      # add up all the lengths along i->k until l
      length = 0
      child = i
      while True:
        length += T[parant[child], child]
        if length > l:
          return parant[child], child, length-l # length-l is the distance from parant[child] to new node
        child = parant[child]
    else:
      for w in np.nonzero(T[v])[0]:
        if parant[w] == -1:
          Q.append(w)
          parant[w] = v
  return dis.astype(int)


def add_phylogeny(dis, n):

  m = dis.shape[0]-1

  if n == 2:
    T = np.zeros((2*m, 2*m), dtype='int32')
    T[0, 1] = dis[0, 1]
    T[1, 0] = dis[1, 0]
    return T

  n -= 1
  limb_len = limb(dis[:n+1,:n+1], n)
  dis[:n, n] -= limb_len
  dis[n, :] = dis[:, n]
  
  # find i, n, k satisfying dis[i, k] == dis[i, n] + dis[n, k]
  # u, v are nearest two nodes of n
  found = False
  for i in range(n):
    for k in range(n):
      if i != k and dis[i, k] == dis[i, n] + dis[n, k]:
        T = add_phylogeny(dis, n)
        u, v, d = nearest(T, dis[i, n], i, k)
        found = True
      if found: break
    if found: break

  # add internal node (n+m-1)
  T[u, n+m-1] = d
  T[n+m-1, u] = d
  T[v, n+m-1] = T[u, v] - d
  T[n+m-1, v] = T[v, n+m-1]
  # reset the distance u-v
  T[u, v] = 0
  T[v, u] = 0
  # add a leaf(n) to the internal node(n+m-1)
  T[n, n+m-1] = limb_len
  T[n+m-1, n] = limb_len

  return T

def UPGMA(dis):

  n = len(dis)
  lable = np.arange(n) # lable of rows and cols of dis matrix
  T = np.zeros((2*n-1, 2*n-1))
  depth = np.zeros(2*n-1)
  proportion = np.ones(2*n-1) # # of nodes in a certain cluster

  while dis.shape[0] > 1:

    # i and j are the indices to be merged
    min_idx = np.argmin(dis[dis.nonzero()])
    i = min_idx // (dis.shape[0]-1)
    j = min_idx % (dis.shape[0]-1)
    if i <= j:
      j += 1
    
    proportion[n] = proportion[lable[i]] + proportion[lable[j]]
    cluster = (dis[i]*proportion[lable[i]] + dis[j]*proportion[lable[j]])/proportion[n]
    depth[n] = dis[i,j]/2
    
    T[lable[i], n] = depth[n] - depth[lable[i]]
    T[lable[j], n] = depth[n] - depth[lable[j]]
    T[n, lable[i]] = T[lable[i], n]
    T[n, lable[j]] = T[lable[j], n]
    
    # update dis matrix
    cluster = np.delete(cluster, [i, j])
    dis = np.delete(dis, [i, j], axis=0)
    dis = np.delete(dis, [i, j], axis=1)
    dis = np.column_stack((cluster, dis))
    cluster = np.insert(cluster, 0, 0)
    dis = np.row_stack((cluster, dis))

    # update lable
    lable = np.delete(lable, [i, j])
    lable = np.insert(lable, 0, n)
    
    n += 1
  
  return T

def neighbor_join(dis):

  n = len(dis)
  lable = np.arange(n) # lable of rows and cols of dis matrix
  T = np.zeros((2*(n-1), 2*(n-1)))

  while dis.shape[0] > 2:

    # neighbor-joining matrix constructed from dis
    Q = dis*(dis.shape[0]-2) - dis.sum(axis=0) - dis.sum(axis=1, keepdims=True)
    Q[np.arange(Q.shape[0]), np.arange(Q.shape[0])] = 0 # diagonal element = 0

    # i and j are the indices to be merged
    min_idx = np.argmin(Q[Q.nonzero()])
    i = min_idx // (Q.shape[0]-1)
    j = min_idx % (Q.shape[0]-1)
    if i <= j:
      j += 1
    
    delta = (dis[i].sum() - dis[j].sum())/(Q.shape[0]-2)

    T[lable[i], n] = (dis[i,j] + delta)/2 # limb_length of i
    T[lable[j], n] = (dis[i,j] - delta)/2 # limb_length of j
    T[n, lable[i]] = T[lable[i], n]
    T[n, lable[j]] = T[lable[j], n]

    # update dis matrix
    new = (dis[i] + dis[j] - dis[i,j])/2
    new = np.delete(new, [i, j])
    dis = np.delete(dis, [i, j], axis=0)
    dis = np.delete(dis, [i, j], axis=1)
    dis = np.column_stack((new, dis))
    new = np.insert(new, 0, 0)
    dis = np.row_stack((new, dis))

    # update lable
    lable = np.delete(lable, [i, j])
    lable = np.insert(lable, 0, n)
    
    n += 1

  T[lable[0], lable[1]] = dis[0, 1]
  T[lable[1], lable[0]] = dis[1, 0]

  return T


# Small Parsimony Problem
class Node:

  def __init__(self):
    self.data = ''
    self.left = None
    self.right = None

  def __str__(self):
    return str(self.data)

class Tree:

  def __init__(self, root):
    self.root = root
    self.len = None # string length

  def insert(self, node, child):
    # assign string length
    if self.len == None:
      if child.data != '':
        self.len = len(child.data)
    if node.left == None:
      node.left = child
    else:
      node.right = child

  def small_parsimony(self):

    D = {'A':0, 'C':1, 'G':2, 'T':3}
    ACGT = 'ACGT'

    level_order = []
    Q = collections.deque()
    Q.append(self.root)
    while Q:
      node = Q.popleft()
      level_order.append(node)
      if node.left != None: Q.append(node.left)
      if node.right != None: Q.append(node.right)

    n = (len(level_order)+1)//2 # n = # of leaves
    total_s = 0

    for i in range(self.len):

      s = np.zeros((2*n-1, 4))
      s[n-1:] = float('inf') # s of leaves are set inf initially
      for j in range(n-1, 2*n-1):
        s[j, D[level_order[j].data[i]]] = 0
      
      # upward
      for j in reversed(range(n-1)):
        for base in ('A','C','G','T'):
          left = s[2*j+1] + 1
          left[D[base]] -= 1
          right = s[2*j+2] + 1
          right[D[base]] -= 1
          s[j, D[base]] = min(left) + min(right)

      # downward
      for j in range(n-1):
        min_idx = np.argmin(s[j])
        s[2*j+1] += 1 # left
        s[2*j+1, min_idx] -= 1
        s[2*j+2] += 1 # right
        s[2*j+2, min_idx] -= 1
        level_order[j].data += ACGT[min_idx]

      total_s += min(s[0])
    

    print(total_s)
    for i in range(n-1):
      ham_left = ham(level_order[i].data, level_order[2*i+1].data)
      ham_right = ham(level_order[i].data, level_order[2*i+2].data)
      print(level_order[i], '->', level_order[2*i+1], ':', ham_left, sep='')
      print(level_order[2*i+1], '->', level_order[i], ':', ham_left, sep='')
      print(level_order[i], '->', level_order[2*i+2], ':', ham_right, sep='')
      print(level_order[2*i+2], '->', level_order[i], ':', ham_right, sep='')

    # n = f.readline()
    # n = int(n)
    # lines = f.readlines()
    # node = []
    # for i in range(2*n-1):
    #   node.append(Node())
    # T = Tree(node[-1])
    # for i in range(n):
    #   idx, data = re.split('->', lines[i].strip())
    #   idx = int(idx)
    #   node[i].data = data
    #   T.insert(node[idx], node[i])
    # for i in range(n, len(lines)):
    #   idx, idx_child = re.split('->', lines[i].strip())
    #   idx, idx_child = int(idx), int(idx_child)
    #   T.insert(node[idx], node[idx_child])
    # T.small_parsimony()

Chapter 8

In [ ]:
def far_first_traversal(data, k):

  centers = np.array([data[0]])
  while len(centers) < k:
    max_distance = 0
    for point in data:
      distance = min(np.sqrt(np.sum(np.square(centers-point), axis=1)))
      if distance > max_distance:
        max_distance = distance
        data_point = point
    centers = np.vstack((centers, np.array([data_point])))
  return centers

def distortion(data, centers):

  distance = 0

  for point in data:
    distance += min(np.sum(np.square(centers-point), axis=1))

  distance /= len(data)

  return distance

def lloyd(data, k):

  centers = data[:k]

  while True:

    clusters = []
    for i in range(k):
      clusters.append([])

    for point in data:
      idx = np.argmin(np.sum(np.square(centers-point), axis=1))
      clusters[idx].append(point)
    
    new_centers = []
    for cluster in clusters:
      new_centers.append(np.array(cluster).sum(axis=0)/len(cluster))
    new_centers = np.array(new_centers)

    if np.array_equal(centers, new_centers):
      break
    else:
      centers = new_centers
  
  return centers.round(3)

def soft_cluster(data, k, m, stiff):

  centers = data[:k]

  for run in range(100):
    distance = np.sqrt(np.sum(np.square(centers.reshape(k, 1, m) - data.reshape(1, -1, m)), axis=2))
    hidden = np.exp(-stiff*distance)/np.sum(np.exp(-stiff*distance), axis=0)
    centers = (hidden @ data)/np.sum(hidden, axis=1, keepdims=True)

  return centers.round(3)



f = open('/content/drive/My Drive/Colab Notebooks/input.txt', 'r')
k, m = f.readline().split()
k, m = int(k), int(m)
stiff = f.readline()
stiff = float(stiff)
data = []
centers = []
# for i in range(k):
#   centers.append(list(map(float, f.readline().split())))
# f.readline()
for line in f.readlines():
  data.append(list(map(float, line.strip().split())))
data = np.array(data)
# centers = np.array(centers)
for i in soft_cluster(data, k, m, stiff):
  print(*i)

4.239 7.764 19.051
4.73 11.406 4.662
13.069 5.979 13.022
18.252 7.745 4.501
7.363 20.105 7.091
6.422 3.779 3.362
3.846 3.909 9.867


In [ ]:
!a = 'abc'
print(a[1])

b
